# Laboratorio 7
### Oscar Fernando López Barrios
### Carné 20679

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/malimg_paper_dataset_imgs"

In [15]:
!pip install adversarial-robustness-toolbox

### Ataque 1: Fast Gradient Method

El modelo de Fast Gradient Method es una técnica fundamental en el campo del aprendizaje automático, especialmente cuando se trata de evaluar la robustez de los modelos en entornos adversarios.

Este método nos permite evaluar qué tan resistente es nuestro modelo ante estos ataques manipulativos. Funciona generando pequeñas perturbaciones en los datos de entrada, imperceptibles para los humanos, pero que pueden causar que el modelo clasifique incorrectamente la imagen.

Esta capacidad para generar imágenes "adversarias" nos ayuda a entender las debilidades de nuestro modelo y nos permite fortalecerlo. Al evaluar cómo responde nuestro modelo a estas perturbaciones, podemos identificar áreas que necesitan mejorar y desarrollar estrategias para hacer que el modelo sea más robusto frente a tales ataques.

En resumen, el Fast Gradient Method es una herramienta esencial para garantizar que nuestros modelos de aprendizaje automático sean confiables y seguros en entornos donde la seguridad es primordial, como en sistemas de reconocimiento de imágenes en aplicaciones médicas o de seguridad.

In [19]:
import numpy as np
from tensorflow.keras.models import load_model
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

modelo = load_model('modelo_lab_7.h5')

classifier = KerasClassifier(model=modelo, clip_values=(0, 1))

fsgm_attack = FastGradientMethod(estimator=classifier, eps=0.1)

data = ImageDataGenerator(rescale=1./255)
test = data.flow_from_directory(path,
    target_size=(64, 64),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

x_test = []
y_data = []
for i in range(len(test)):
    x, y = test[i]
    x_adv = fsgm_attack.generate(x=x)
    x_test.append(x_adv[0]) 
    y_data.append(y[0])

y_pred = classifier.predict(np.array(x_test))

y_pred_indices = np.argmax(y_pred, axis=1)
y_data_indices = np.argmax(y_data, axis=1)

accuracy = np.mean(y_pred_indices == y_data_indices)
print("Accuracy:", accuracy)

precision = np.sum(y_pred_indices == y_data_indices) / len(y_data_indices)
print("Precision:", precision)

Found 9339 images belonging to 25 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Accuracy: 0.08234286326159118
Precision: 0.08234286326159118


### Ataque 2: DeepFool
DeepFool es una excelente herramienta para evaluar cuán robusto es nuestro modelo de aprendizaje automático frente a pequeñas perturbaciones en los datos de entrada. Esto es clave, especialmente en aplicaciones donde incluso pequeños cambios en los datos pueden tener un gran impacto en las decisiones del modelo.

Mediante el análisis de cómo nuestro modelo responde a las perturbaciones generadas por DeepFool, podemos obtener una comprensión más profunda de sus decisiones y límites. Esta comprensión nos permite identificar áreas de mejora y realizar ajustes para fortalecer nuestro modelo.

En conclusión, DeepFool es una herramienta valiosa que nos permite evaluar la robustez y seguridad de nuestros modelos de aprendizaje automático, mejorar su capacidad de generalización y comprensión, y garantizar su eficacia en aplicaciones críticas.

In [20]:
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import DeepFool

model_logits = tf.keras.Model(inputs=modelo.input, outputs=modelo.layers[-2].output)

classifier = KerasClassifier(model=model_logits, use_logits=True, clip_values=(0, 1))

attack = DeepFool(classifier=classifier)

x_test, y_test = next(test)
x_test_adv = attack.generate(x=x_test)

y_pred_adv = classifier.predict(x_test_adv)

y_pred_adv_indices = np.argmax(y_pred_adv, axis=1)
y_true_indices = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_adv_indices == y_true_indices)
print("Accuracy:", accuracy)

precision = np.sum(y_pred_adv_indices == y_true_indices) / len(y_true_indices)
print("Precision:", precision)

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 0.0
Precision: 0.0
